In [1]:
from Nexullance_IT import Nexullance_IT
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from topologies.DDF import DDFtopo
import globals as gl
import numpy as np
import copy
import time


In [2]:
config=(36, 5)
EPR=3
ddf_network = DDFtopo(config[0], config[1])

In [3]:
Cap_remote = 10 #GBps
Cap_local = 10 #GBps
M_EPs = gl.generate_uniform_traffic_pattern(config[0], EPR) # an numpy matrix filled with ones
scaling_factor = 1
M_EPs = scaling_factor * M_EPs
M_R = gl.convert_M_EPs_to_M_R(M_EPs, config[0], EPR)

ASP = ddf_network.calculate_all_shortest_paths()
ECMP_ASP = gl.ECMP(ASP)
remote_link_flows, local_link_flows = ddf_network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EPs)
max_remote_link_load = np.max(remote_link_flows)/Cap_remote
max_local_link_load = np.max(local_link_flows)/Cap_local
print("Max remote link load: ", max_remote_link_load)
print("Max local link load: ", max_local_link_load)
if max_remote_link_load > 1.0 or max_local_link_load > 1.0:
    print("network congestion may present")
    mu = max_local_link_load/max_remote_link_load
    if mu > 1.0:
        print(f"mu = {mu} > 1, max local link load is higher")
    else:
        print(f"mu = {mu} < 1, max remote link load is higher")
else:
    print("network is probably not congested")
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, max_remote_link_load, max_local_link_load), "GBps")


In [5]:
nexu_it = Nexullance_IT(ddf_network.nx_graph, M_R, Cap_remote)

In [6]:
nexu_it.initialize()

# debug method 1 (RAM and time for multiple invocations)

In [7]:
def test_weight_function( s: int, d: int, edge_attributes: dict):
    # define the weight function for the dijkstra or the bellman-ford algorithm.
    alpha: float = 10.0
    beta: float = 1.0
    return alpha+edge_attributes['load']**beta


20.77500000000001

In [ ]:
nexu_it_test = copy.deepcopy(nexu_it)
nexu_it_test.optimization_method_1(False, test_weight_function)